# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda x, y: lambda t: y(f(t, x)), lambda x: x, lst)(x0)

In [6]:
def foldr2(f, x0, lst):
    return foldl(lambda y, x: lambda t: y(f(x, t)), lambda x: x, lst)(x0)

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [12]:
def chars_range(str1,str2):
    ord_range = range(ord(str1),ord(str2))
    for number in ord_range:
        yield chr(number)

def check_inv(a, b):
    b_chars = dict.fromkeys(chars_range('a','z'),0)
    for char in b:
        b_chars[char] +=1
       
    curr_chars = dict.fromkeys(chars_range('a','z'),0)
    for char in a[:len(b)-1]:
        curr_chars[char] += 1
      
    for i in range(len(b) - 1, len(a)):
        curr_chars[a[i]] += 1
        
        if (curr_chars==b_chars):
            return True
        curr_chars[a[i-len(b)+1]] -=1
    return False



a = 'abcrotm'
b = 'tro'
check_inv(a, b)

True

# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [8]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __iter__(self):
        if not self.left and not self.right:
            yield self.value
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right

    def __str__(self):
        return f'{self.value}({self.left},{self.right})'.replace('None', '-').replace('(-,-)', '')

    def __repr__(self):
        return '{}({}, {}, {})'.format(self.__class__.__name__,
                                       repr(self.value),
                                       repr(self.left) if self.left else '-',
                                       repr(self.right) if self.right else '-').replace(', -', '')
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [2]:
def calc(expr):
    value = ''
    stack = []
    sign = []
    bracket = []
    i = 0
    while i < len(expr):
        x = expr[i]
        if x == ' ':
            pass
        elif x in '1234567890':
            value += x
        elif x in '+-*/':
            if x == '*' and expr[i+1] == '*':
                x = '^'
                i += 1
            if value == '' and x == '-':
                pass
            else:
                stack.append(int(value))
            sign.append(x)
            value = ''
        elif x == '(':
            bracket.append(i)
            for j in range(len(expr))[i+1::]:
                if expr[j] == ')':
                    bracket.pop()
                    if len(bracket) == 0:
                        value = calc(expr[i + 1:j])
                        i = j
                        break
                if expr[j] == '(':
                    bracket.append(i)
        i += 1
    stack.append(int(value))
    value = ''
    i = 0
    if len(stack) == len(sign) and sign[0] == '-':
        stack[0] = -stack[0]
        sign.pop(0)
    while len(stack) > 1:
        if '^' in sign:
            if sign[i] == '^':
                stack[i] = stack[i] ** stack[i+1]
                stack.pop(i+1)
                sign.pop(i)
                i -= 1
                if '^' not in sign:
                    i = -1
        elif '*' in sign or '/' in sign:
            if sign[i] == '*':
                stack[i] = stack[i] * stack[i+1]
                stack.pop(i+1)
                sign.pop(i)
                i -= 1
                if '*' not in sign and '/' not in sign:
                    i = -1
            elif sign[i] == '/':
                stack[i] = stack[i] / stack[i+1]
                stack.pop(i+1)
                sign.pop(i)
                i -= 1
                if '*' not in sign and '/' not in sign:
                    i = -1
        else:
            if sign[i] == '+':
                stack[i] = stack[i] + stack[i+1]
            elif sign[i] == '-':
                stack[i] = stack[i] - stack[i+1]
            stack.pop(i+1)
            sign.pop(i)
            i -= 1
        i += 1
    return stack[0]


calc('2 * (15 - 3 * 4) - 2') == 4

True